In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image

import click
import numpy as np
import pandas as pd
import scipy.io
import sys
import tqdm

from PIL import Image, ImageFilter
from tensorboardX import SummaryWriter


from models.deeplabv2 import DeepLabV2
from models.msc import MSC
from models.discriminator import Discriminator
from dataset import PartAffordanceDataset, PartAffordanceDatasetWithoutLabel
from dataset import CenterCrop, ToTensor, Normalize



''' one-hot representation '''

def one_hot(label, n_classes, device):
    one_hot_label = torch.eye(n_classes, requires_grad=True, device=device)[label].transpose(1, 3).transpose(2, 3)
    return one_hot_label
    

''' scheduler for learning rate '''

def poly_lr_scheduler(optimizer, init_lr, iter, lr_decay_iter, max_iter, power):
    if iter % lr_decay_iter or iter > max_iter:
        return None
    new_lr = init_lr * (1 - float(iter) / max_iter) ** power
    optimizer.param_groups[0]["lr"] = new_lr
    optimizer.param_groups[1]["lr"] = 10 * new_lr
    optimizer.param_groups[2]["lr"] = 20 * new_lr


def poly_lr_scheduler_d(optimizer, init_lr, iter, lr_decay_iter, max_iter, power):
    if iter % lr_decay_iter or iter > max_iter:
        return None
    new_lr = init_lr * (1 - float(iter) / max_iter) ** power
    optimizer.param_groups[0]["lr"] = new_lr
    if len(optimizer.param_groups) > 1:
        optimizer.param_groups[1]['lr'] = 10 * new_lr


''' model, weight initialization, get params '''

def init_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        if m.bias is not None:
            nn.init.constant_(m.weight, 1)


def DeepLabV2_ResNet101_MSC(n_classes):
    return MSC(
        scale=DeepLabV2(
            n_classes=n_classes, n_blocks=[3, 4, 23, 3], pyramids=[6, 12, 18, 24]
        ),
        pyramids=[0.5, 0.75],
    )


def get_params(model, key):
    # For Dilated FCN
    if key == "1x":
        for m in model.named_modules():
            if "layer" in m[0]:
                if isinstance(m[1], nn.Conv2d):
                    for p in m[1].parameters():
                        yield p
    # For conv weight in the ASPP module
    if key == "10x":
        for m in model.named_modules():
            if "aspp" in m[0]:
                if isinstance(m[1], nn.Conv2d):
                    yield m[1].weight
    # For conv bias in the ASPP module
    if key == "20x":
        for m in model.named_modules():
            if "aspp" in m[0]:
                if isinstance(m[1], nn.Conv2d):
                    yield m[1].bias



''' training '''

def full_train(
        model, model_d, sample, criterion_ce_full, criterion_bce, 
        optimizer, optimizer_d, ones, zeros, device):

    ''' full supervised learning '''
    
    model.train()
    model.scale.freeze_bn()
    model_d.train()

    # train segmentation network
    x, y = sample['image'], sample['class']

    batch_len = len(x)

    x = x.to(device)
    y = y.to(device)

    h = model(x)     # shape => (N, 8, H/8, W/8)
    h = F.interpolate(h, size=(256, 256), mode='bilinear', align_corners=True)

    h_ = h.detach()    # h_ is for calculating loss for discriminator
    y_ = y.detach()    # y_is for the same purpose.  shape => (N, H, W)

    d_out = model_d(h)    # shape => (N, 1, H/32, W/32)
    d_out = F.interpolate(d_out, size=(256, 256), mode='bilinear', align_corners=True)    # shape => (N, 1, H, W)
    d_out = d_out.squeeze()
    
    loss_ce = criterion_ce_full(h, y)
    loss_adv = criterion_bce(d_out, ones[:batch_len])
    loss_full = loss_ce + 0.01 * loss_adv

    optimizer.zero_grad()
    optimizer_d.zero_grad()
    loss_full.backward()
    optimizer.step()


    # train discriminator
    seg_out = model_d(h_)    # shape => (N, 1, H/32, W/32)
    seg_out = F.interpolate(seg_out, size=(256, 256), mode='bilinear', align_corners=True)    # shape => (N, 1, H, W)
    seg_out = seg_out.squeeze()
    
    y_ = one_hot(y_, 8, device)    # shape => (N, 8, H, W)
    true_out = model_d(y_)    # shape => (N, 1, H/32, W/32)
    true_out = F.interpolate(true_out, size=(256, 256), mode='bilinear', align_corners=True)    # shape => (N, 1, H, W)
    true_out = true_out.squeeze()

    loss_d_fake = criterion_bce(seg_out, zeros[:batch_len])
    loss_d_real = criterion_bce(true_out, ones[:batch_len])
    loss_d = loss_d_fake + loss_d_real

    optimizer.zero_grad()
    optimizer_d.zero_grad()
    loss_d.backward()
    optimizer_d.step()
    
    return loss_full.item(), loss_d.item()




def semi_train(
        model, model_d, sample, criterion_ce_semi, criterion_bce, 
        optimizer, optimizer_d, ones, zeros, device):

    ''' semi supervised learning '''
    
    model.train()
    model.scale.freeze_bn()
    model_d.eval()

    # train segmentation network
    x = sample['image']
    batch_len = len(x)
    
    x = x.to(device)
    
    h = model(x)     # shape => (N, 8, H/8, W/8)
    h = F.interpolate(h, size=(256, 256), mode='bilinear', align_corners=True)

    _, h_ = torch.max(h, dim=1)    # to calculate the crossentropy loss. shape => (N, H, W)

    with torch.no_grad():
        d_out = model_d(h)    # shape => (N, 1, H/32, W/32)
        d_out = F.interpolate(d_out, size=(256, 256), mode='bilinear', align_corners=True)    # shape => (N, 1, H, W)
        d_out = d_out.squeeze()

    loss_adv = criterion_bce(d_out, ones[:batch_len])


    # if the pixel value of the output from discriminator is more than a threshold,
    # its value is viewd as one from true label. Else, its value is ignored(value=255).
    h_[d_out < 0.2] = 255

    loss_ce = criterion_ce_semi(h, h_)
    loss_semi = 0.001 * loss_adv + 0.1 * loss_ce

    optimizer.zero_grad()
    optimizer_d.zero_grad()
    loss_semi.backward()
    optimizer.step()

    return loss_semi.item()

In [2]:
pretrained_model = './models/deeplabv2_resnet101_COCO_init.pth'
class_weight_flag = True
batch_size = 6
num_workers = 4
max_epoch = 1000
learning_rate = 0.00025
learning_rate_d = 0.0001
n_classes = 8
device = 'cuda'

In [3]:
''' DataLoader '''
train_data_with_label = PartAffordanceDataset('train_with_label.csv',
                                        transform=transforms.Compose([
                                            CenterCrop(),
                                            ToTensor(),
                                            Normalize()
                                        ]))

train_data_without_label = PartAffordanceDatasetWithoutLabel('train_without_label.csv',
                                        transform=transforms.Compose([
                                            CenterCrop(),
                                            ToTensor(),
                                            Normalize()
                                         ]))

test_data = PartAffordanceDataset('test.csv',
                            transform=transforms.Compose([
                                CenterCrop(),
                                ToTensor(),
                                Normalize()
                            ]))

train_loader_with_label = DataLoader(train_data_with_label, batch_size=batch_size, shuffle=True, num_workers=num_workers)
train_loader_without_label = DataLoader(train_data_without_label, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)


''' define model, optimizer, loss '''
model = DeepLabV2_ResNet101_MSC(n_classes)
model_d = Discriminator(n_classes)

model.apply(init_weights)
model_d.apply(init_weights)

state_dict = torch.load(pretrained_model)
model.load_state_dict(state_dict, strict=False)

model.to(device)
model_d.to(device)

optimizer = optim.SGD(
                    params=[{
                        "params": get_params(model, key="1x"),
                        "lr": learning_rate,
                        "weight_decay": 5.0e-4,
                            },
                            {
                        "params": get_params(model, key="10x"),
                        "lr": 10 * learning_rate,
                        "weight_decay": 5.0e-4,
                            },
                            {
                            "params": get_params(model, key="20x"),
                            "lr": 20 * learning_rate,
                            "weight_decay": 0.0,
                            }],
                    momentum=0.9)



optimizer_d = optim.Adam(model_d.parameters(), lr=learning_rate_d, betas=(0.9,0.99))

if class_weight_flag:
    class_weight = torch.tensor([0.0057, 0.4689, 1.0000, 1.2993, 
                                0.4240, 2.3702, 1.7317, 0.8149])    # refer to dataset.py
    criterion_ce_full = nn.CrossEntropyLoss(weight=class_weight.to(device))
else:
    criterion_ce_full = nn.CrossEntropyLoss()

criterion_ce_semi = nn.CrossEntropyLoss(ignore_index=255)
criterion_bce = nn.BCEWithLogitsLoss()

# supplementary constant for discriminator
ones = torch.ones(batch_size, 256, 256).to(device)
zeros = torch.zeros(batch_size, 256, 256).to(device)


''' training '''

losses_full = []
losses_semi = []
losses_d = []
val_iou = []
mean_iou = []
best_iou = 0.0

In [4]:
for epoch in range(max_epoch):

    epoch_loss_full = 0.0
    epoch_loss_d = 0.0
    epoch_loss_semi = 0.0

    poly_lr_scheduler(
        optimizer=optimizer,
        init_lr=learning_rate,
        iter=epoch - 1,
        lr_decay_iter=10,
        max_iter=max_epoch,
        power=0.9,
    )

    poly_lr_scheduler_d(
        optimizer=optimizer_d,
        init_lr=learning_rate_d,
        iter=epoch - 1,
        lr_decay_iter=10,
        max_iter=max_epoch,
        power=0.9,
    )


    # only supervised learning
    if epoch < 0:
        for i, sample in tqdm.tqdm(enumerate(train_loader_with_label), 
                                   total=len(train_loader_with_label)):

            loss_full, loss_d = full_train(
                                    model, model_d, sample, criterion_ce_full, criterion_bce,
                                    optimizer, optimizer_d, ones, zeros, device)
            print(loss_full, loss_d)
            epoch_loss_full += loss_full
            epoch_loss_d += loss_d

        losses_full.append(epoch_loss_full / i)   # mean loss over all samples
        losses_d.append(epoch_loss_d / i)
        losses_semi.append(0.0)


    # semi-supervised learning
    if epoch >= 0:
        for i, (sample1, sample2) in tqdm.tqdm(enumerate(zip(train_loader_with_label, train_loader_without_label)), 
                                                total=len(train_loader_with_label)):

            loss_full, loss_d = full_train(
                                    model, model_d, sample1, criterion_ce_full, criterion_bce,
                                    optimizer, optimizer_d, ones, zeros, device)
            
            epoch_loss_full += loss_full
            epoch_loss_d += loss_d

            loss_semi = semi_train(
                                    model, model_d, sample2, criterion_ce_semi, criterion_bce,
                                    optimizer, optimizer_d, ones, zeros, device)
            
            epoch_loss_semi += loss_semi

        losses_full.append(epoch_loss_full / i)   # mean loss over all samples
        losses_d.append(epoch_loss_d / i)
        losses_semi.append(epoch_loss_semi / i)

  0%|          | 0/1967 [00:00<?, ?it/s]/home/yuchi/anaconda3/envs/torch/lib/python3.5/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


2.8698792457580566 1.253151297569275


  0%|          | 1/1967 [00:01<41:05,  1.25s/it]

0.011708456091582775
11.97008991241455 4.213683128356934


  0%|          | 2/1967 [00:02<40:32,  1.24s/it]

...

0.027784625068306923
2.2203335762023926 0.39545154571533203


  5%|▌         | 100/1967 [01:57<36:24,  1.17s/it]Process Process-1:
Process Process-2:
Process Process-5:
Process Process-4:
Process Process-8:
Process Process-7:
Process Process-3:
Process Process-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/multip

  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
KeyboardInterrupt
  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt
KeyboardInterrupt
  File "/home/yuchi/anaconda3/envs/torch/lib/python3.5/multiprocessing/queues.py", line 104, in get
    if timeout 

0.050244495272636414


KeyboardInterrupt: 